# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [ ]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [12]:
# TODO: Import the necessary libs
import os
from typing import List, Dict
import json
import chromadb
from chromadb.utils import embedding_functions
from lib.agents import Agent
from lib.llm import LLM
from pydantic import BaseModel
from lib.tooling import tool
from tavily import TavilyClient
from dotenv import load_dotenv, find_dotenv

In [10]:
# TODO: Load environment variables
path = find_dotenv(usecwd=True)
load_dotenv(dotenv_path=path, override=True)
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

True

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [28]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(api_key=os.getenv("OPENAI_API_KEY"))
chroma_client = chromadb.PersistentClient(path="chromadb")
collection = chroma_client.get_collection(
    name="udaplay",
    embedding_function=embedding_fn
)

# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game

@tool
def retrieve_game(query: str):
    """
    Semantic search: Finds most results in the vector DB
    args:
        query: a question about game industry.
    """
    result = collection.query(
        query_texts=[query],
        n_results=5,
        include=["metadatas"]
    )

    metadatas = result.get("metadatas", [[]])[0]

    games = []
    for meta in metadatas:
        games.append({
            "Platform": meta.get("Platform"),
            "Name": meta.get("Name"),
            "YearOfRelease": meta.get("YearOfRelease"),
            "Description": meta.get("Description"),
        })

    return games

#### Evaluate Retrieval Tool

In [19]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result

judge = LLM(model="gpt-4o-mini")

class EvaluationReport(BaseModel):
    useful: bool
    description: str

@tool
def evaluate_retrieval(question: str, retrieved_docs: List[Dict]) -> dict:
    """
    Based on the user's question and on the list of retrieved documents,
    it will analyze the usability of the documents to respond to that question.
    args:
        question: original question from user
        retrieved_docs: retrieved documents most similar to the user query in the Vector Database
    """
    docs_text = json.dumps(retrieved_docs, indent=2)

    prompt = f"""
Determine if the following retrieved documents are sufficient to answer the user question.

User Question:
{question}

Retrieved Documents:
{docs_text}

Provide a detailed explanation.
"""

    ai_message = judge.invoke(
        input=prompt,
        response_format=EvaluationReport
    )

    report = EvaluationReport.model_validate_json(ai_message.content)

    return report.model_dump()

#### Game Web Search Tool

In [48]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry.

tavily = TavilyClient()

@tool
def game_web_search(question: str) -> List[Dict]:
    """
    Performs a real-world web search to gather factual and recent information
    and to verify details not available in the vector database.
    args:
        question: a question about game industry.
    """
    response = tavily.search(
        query=question,
        n_tokens=2048,
        max_results=5
    )

    print(f"[GameWebSearch] Question: {question}")

    results = []
    for item in response.get("results", []):
        results.append({
            "title": item.get("title"),
            "url": item.get("url"),
            "content": item.get("content"),
        })

    return results

### Agent

In [51]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed

GAME_AGENT_INSTRUCTIONS = """
You are a Game Industry Research Agent.

Your workflow:
1. Understand the user's question about video games.
2. Always follow this tool sequence:
   a. Call retrieve_game to get vector-database results.
   b. Call evaluate_retrieval to check whether the DB results fully answer the question.
   c. If the DB results are incomplete, ambiguous, outdated, or if the question requires factual verification (dates, platforms, release details, announcements, sales, or updates), you MUST call game_web_search to obtain recent and accurate information.
3. Use only information returned by tools. Do not guess or hallucinate.
4. After tools provide enough information, give a clear and correct final answer.
5. If the vector DB and the web disagree, ALWAYS trust the web search.
"""

game_agent = Agent(
    model_name="gpt-4o",
    temperature=0.0,
    instructions=GAME_AGENT_INSTRUCTIONS,
    tools=[retrieve_game, evaluate_retrieval, game_web_search],
)


In [52]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?

questions = [
    "When Pokémon Gold and Silver was released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X released for Playstation 5?",
    "When was Cyberpunk 2077 released and for which platforms was it originally launched?"
]

for q in questions:
    run = game_agent.invoke(q)
    final_state = run.get_final_state()

    print("QUESTION:", q)
    print("ANSWER:", final_state["messages"][-1].content)
    print("-----")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
QUESTION: When Pokémon Gold and Silver was released?
ANSWER: Pokémon Gold and Silver was released in 1999 for the Game Boy Color.
-----
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
QUESTION: Which one was the first 3D platformer Mario game?
ANSWER: The first 3D platformer Mario game was "Super Mario 64," released for the Nintendo 64 in 1996.
-----
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[GameWebSearch] Question: Was Mortal Kombat X released 

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes